In [1]:
# Import Splinter, BeautifulSoup, Regular Expression, and Time
from splinter import Browser
from bs4 import BeautifulSoup
import re
import time

In [2]:
# Path to chromedriver
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
# Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path)

In [4]:
# Visit the mars nasa news site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [5]:
# Convert the browser html to a soup object and then quit the browser
html = browser.html
news_soup = BeautifulSoup(html, 'html.parser')

slide_elem = news_soup.select_one('ul.item_list li.slide')

In [6]:
slide_elem.find("div", class_='content_title')

<div class="content_title"><a href="/news/8641/nasas-perseverance-mars-rover-gets-its-wheels-and-air-brakes/" target="_self">NASA's Perseverance Mars Rover Gets Its Wheels and Air Brakes</a></div>

### Get Latest New Title

In [12]:
# Use the parent element to find the first a tag and save it as `news_title`
title = slide_elem.find("div", class_='content_title').get_text()
title

"NASA's Perseverance Mars Rover Gets Its Wheels and Air Brakes"

### Paragraph Text From Corresponding News Title

In [13]:
# Use the parent element to find the paragraph text
p_text = slide_elem.find('div', class_="article_teaser_body").get_text()
p_text

'After the rover was shipped from JPL to Kennedy Space Center, the team is getting closer to finalizing the spacecraft for launch later this summer.'

## JPL Mars Space Images - Featured Image

In [17]:
# Visit URL
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [18]:
# Find and click the full image button
f_image_button = browser.find_by_id('full_image')
f_image_button.click()

In [19]:
# Find the more info button and click that
browser.is_element_present_by_text('more info', wait_time=1)
more_info_elem = browser.find_link_by_partial_text('more info')
more_info_elem.click()

In [20]:
# Parse the resulting html with beautiful soup
html = browser.html
img_soup = BeautifulSoup(html, 'html.parser')

In [23]:
# find the relative image url
rel_url = img_soup.select_one('figure.lede a img').get("src")
rel_url

'/spaceimages/images/largesize/PIA14579_hires.jpg'

In [26]:
# Use the base url to create an absolute url
featured_img_url = f'https://www.jpl.nasa.gov{rel_url}'
featured_img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA14579_hires.jpg'

### Mars Weather

In [28]:
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)

In [29]:
html = browser.html
weather_s = BeautifulSoup(html, 'html.parser')

In [31]:
# First, find a tweet with the data-name `Mars Weather`
mars_weather_tweet = weather_s.find('div', attrs={"class": "tweet", "data-name": "Mars Weather"})

In [33]:
# Next, search within the tweet for the p tag or span tag containing the tweet text
# As Twitter is frequently making changes the try/except will identify the tweet
# text usings a regular expression pattern that includes the string 'sol' if there
# is no p tag with a class of 'tweet-text'
try:
    mars_weather = mars_weather_tweet.find("p", "tweet-text").get_text()
    mars_weather

except AttributeError:

    pattern = re.compile(r'sol')
    mars_weather = weather_s.find('span', text=pattern).text
    mars_weather

mars_weather

'InSight sol 481 (2020-04-03) low -93.9ºC (-137.0ºF) high -5.9ºC (21.4ºF)\nwinds from the WNW at 5.3 m/s (11.8 mph) gusting to 18.3 m/s (40.9 mph)\npressure at 6.50 hPa'

### Mars Hemispheres

In [36]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [40]:
hemisphere_image_urls = []

# First, get a list of all of the hemispheres
links = browser.find_by_css("a.product-item h3")

# Next, loop through those links, click the link, find the sample anchor, return the href
for a in range(len(links)):
    hemisphere = {}
    
    # We have to find the elements on each loop to avoid a stale element exception
    browser.find_by_css("a.product-item h3")[a].click()
    
    # Next, we find the Sample image anchor tag and extract the href
    sample_image = browser.find_link_by_text('Sample').first
    hemisphere['img_url'] = sample_image['href']
    
    # Get Hemisphere title
    hemisphere['title'] = browser.find_by_css("h2.title").text
    
    # Append hemisphere object to list
    hemisphere_image_urls.append(hemisphere)
    
    # Finally, we navigate backwards
    browser.back()
    

In [41]:
hemisphere_image_urls

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

### Mars Facts

In [43]:
import pandas as pd
facts = pd.read_html('https://space-facts.com/mars/')[0]
facts.columns=['description', 'value']
facts.set_index('description', inplace=True)
facts

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [44]:
facts.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

In [45]:
browser.quit()

In [ ]:
scrape_